### Function to read dicom images

* Function `get_dcm` reads dicom files from a directory
* It first checks if the DICOM images have `SliceLocation` attribute. If it has, it stores slices according to the `SliceLocation`. Otherwise, it sorts by name.
* Input parameter:
    `dcm_path`: location of dcm data
    
* Output:
    Returns -   
     `dcm_data`: dicom data in a single array <br>
     `pix_spacing`: spacing of pixel/voxel <br>
     `intensity`: min and max intensity <br>

Author @ Mrinal Kanti Dhar

In [1]:
import numpy as np
import os
import pydicom

In [2]:
def get_dcm(dcm_path):
    dicom_img = []
    file_names = []
    # Get DICOM image names
    for dir_name, sub_dir_list, file_list in os.walk(dcm_path):
        for file_name in file_list:
            if ".dcm" in file_name.lower():  # check if it is dcm file 
                file_names.append(file_name)
                dicom_img.append(os.path.join(dir_name,file_name))
    
    # Get ref file
    ref = pydicom.dcmread(dicom_img[0])    
    # Get the spacing
    pix_dim = (int(ref.Rows), int(ref.Columns), len(dicom_img))
    # Get spacing values (in mm)
    pix_spacing = (float(ref.PixelSpacing[0]), float(ref.PixelSpacing[1]), float(ref.SliceThickness))    
    # Get the data
    dcm_data = np.zeros(pix_dim, dtype=ref.pixel_array.dtype)   

    # Check if the DICOM file has 'SliceLocation' attribute. It if it has, then store them
    # according to the slice location. Otherwise, store them chronologically. 
    if hasattr(pydicom.dcmread(dicom_img[0]), 'SliceLocation'):               
        # Sort file names according to the slice location
        sliceLocation = [pydicom.dcmread(dicom_img[i]).SliceLocation for i in range(len(dicom_img))]
        ps = np.argsort(sliceLocation) #ps: position of the sorted slice locations
        sorted_file_names = [file_names[ps[i]] for i in range(len(ps))]
        print('DICOM sorted by slice location')          
        
        # loop through all the DICOM files
        idx = 0
        for file in sorted_file_names:       
            # read the file
            ds = pydicom.dcmread(os.path.join(dir_name,file))
            # store the raw image data
            dcm_data[:, :, idx] = ds.pixel_array
            idx += 1
    else:
        print('No slice location. DICOM sorted by name')

        # loop through all the DICOM files
        for file in dicom_img:
            # read the file
            ds = pydicom.dcmread(os.path.join(dir_name,file))
            # store the raw image data
            dcm_data[:, :, dicom_img.index(file)] = ds.pixel_array                              
    
    # Get the min and max intensity
    intensity = (dcm_data.min(), dcm_data.max())
        
    return dcm_data, pix_spacing, intensity

#### DICOM sorted by `SliceLocation`

In [3]:
dcm_loc1 = 'F:\dicom1'

In [4]:
data, pix_spacing, intensity = get_dcm(dcm_loc1)
print('Dimension: ', data.shape, 
      '\nVoxel spacing: ', pix_spacing, 
      '\nIntensity range: ', intensity)

DICOM sorted by slice location
Dimension:  (400, 400, 280) 
Voxel spacing:  (0.4, 0.4, 0.4) 
Intensity range:  (-1000, 6710)


#### DICOM sorted by name

In [5]:
dcm_loc2 = 'F:\dicom2'

In [6]:
data, pix_spacing, intensity = get_dcm(dcm_loc2)
print('Dimension: ', data.shape, 
      '\nVoxel spacing: ', pix_spacing, 
      '\nIntensity range: ', intensity)

No slice location. DICOM sorted by name
Dimension:  (552, 552, 368) 
Voxel spacing:  (0.25, 0.25, 0.25) 
Intensity range:  (0, 8191)
